This file tries to judge if a galaxy has its mms galaxy on the first infall, but fails.

In [1]:
import os
import gc
from tqdm import tqdm
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import math

# %matplotlib widget
basePath = "./sims.TNG/TNG100-1/output"


In [2]:
initial_snapshot = 69
end_snapshot = 99

halos_fields = ["GroupFirstSub", "GroupNsubs", "GroupMass", "GroupPos"]
subhalos_fields = ["SubhaloGrNr", "SubhaloMass", "SubhaloPos"]
tree_fields = ["SubfindID", "SnapNum"]

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


In [3]:
def Detect_Mkdir(Path):
    if not (os.path.exists(Path)):
        os.makedirs(Path)
    return Path


def Triple_Array_Iter(Data_Array1, Data_Array2, Data_Array3):
    for index in range(0, min(len(Data_Array1), len(Data_Array2), len(Data_Array3))):
        yield Data_Array1[index], Data_Array2[index], Data_Array3[index]


def Calculate_Halo_Dict(halo_index, host_index, mms_index):

    host_tree = il.sublink.loadTree(
        basePath, end_snapshot, host_index, fields=tree_fields, onlyMPB=1)
    mms_tree = il.sublink.loadTree(
        basePath, end_snapshot, mms_index, fields=tree_fields, onlyMPB=1)

    satellite_velocity_list = list()
    host_mms_distance = list()
    host_position_list = list()
    mms_position_list = list()

    for current_snapshot in range(initial_snapshot, end_snapshot+1):

        header = il.groupcat.loadHeader(basePath, current_snapshot)
        scale_factor = header['Time']
        host_subhalo_index = host_tree['SubfindID'][np.where(
            host_tree['SnapNum'] == current_snapshot)[0]]
        mms_subhalo_index = mms_tree['SubfindID'][np.where(
            mms_tree['SnapNum'] == current_snapshot)[0]]

        if len(mms_subhalo_index) == 0 or len(host_subhalo_index) == 0:
            satellite_velocity_list.append(0)
            host_mms_distance.append(0)
            host_position_list.append(np.array([0, 0, 0]))
            mms_position_list.append(np.array([0, 0, 0]))
            continue

        host_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, host_subhalo_index)
        mms_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, mms_subhalo_index)

        current_host_comoving_position = host_subhalo_dict['SubhaloPos']
        current_mms_comoving_position = mms_subhalo_dict['SubhaloPos']

        current_host_physical_position = current_host_comoving_position*scale_factor
        current_mms_physical_position = current_mms_comoving_position*scale_factor

        current_host_physical_position = current_host_physical_position % 35000
        current_mms_physical_position = current_mms_physical_position % 35000

        current_difference_physical_coordinates = current_mms_physical_position - \
            current_host_physical_position
        distance = np.linalg.norm(current_difference_physical_coordinates)

        host_velocity = host_subhalo_dict['SubhaloVel']
        mms_velocity = mms_subhalo_dict['SubhaloVel']

        Vel_diff = mms_velocity-host_velocity
        Vel_Radius = np.dot(
            Vel_diff, current_difference_physical_coordinates)/distance

        satellite_velocity_list.append(Vel_Radius)

        gc.collect()

    satellite_radial_velocity = np.array(satellite_velocity_list)

    return_data = dict()
    return_data['RadialVelocity'] = satellite_radial_velocity
    return_data['Distance'] = host_mms_distance
    return_data['Snapshots'] = np.arange(initial_snapshot, end_snapshot+1)
    return return_data


def Extremum_Calculation(Radial_Velocity_Array):
    Extremum_Number = 0
    for index in np.arange(len(Radial_Velocity_Array)-1):
        if Radial_Velocity_Array[index]*Radial_Velocity_Array[index+1] < 0:
            Extremum_Number += 1
    return Extremum_Number


In [5]:
FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
Final_File_List = ['Halo_Array', 'Host_Array', 'MMS_Array', 'First_App_Snap_Array', 'First_App_Distance_Array',
                   'Snap99_Distance_Array', 'Snap99_Radial_Velocity_Array', 'Snap99_Virial_Radius_Array', 'Is_First_Approach']
for filename in Final_File_List:
    if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
        globals()[filename] = np.load(
            (os.path.join(FilePath, filename)+'.npy'))


In [30]:
FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
filename='Is_First_Infall'
if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
    globals()[filename] = np.load(
        (os.path.join(FilePath, filename)+'.npy'))
else:Is_First_Infall=np.array([])

start_sub_index = Is_First_Infall.shape[0]
step_length = 800
end = start_sub_index+step_length

if end > Halo_Array.shape[0]:end =Halo_Array.shape[0]
Final_File_Is_First_Infall = list()

for Subindex in tqdm(range(start_sub_index,end)):
    Halo_Index=Halo_Array[Subindex]
    Host_Index=Host_Array[Subindex]
    MMS_Index=MMS_Array[Subindex]

    halo_dict=Calculate_Halo_Dict(Halo_Index,Host_Index,MMS_Index)
    If_Extremun_Num_Less_2=(Extremum_Calculation(halo_dict['RadialVelocity'])<=1)
    If_Radial_Velocity_Negative = (
        halo_dict['RadialVelocity'][end_snapshot-initial_snapshot] < 0)
    If_First_Infall = If_Radial_Velocity_Negative and If_Extremun_Num_Less_2
    Final_File_Is_First_Infall.append(If_First_Infall)


Final_File_Is_First_Infall_Array = np.array(Final_File_Is_First_Infall)

if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
    Previous_File = np.load((os.path.join(FilePath, filename)+'.npy'))
    Next_File_Name = 'Final_File_'+filename
    Next_File_Array = globals()[Next_File_Name]
    ConCated_File = np.concatenate(
            (Previous_File, Next_File_Array), axis=0)
    np.save((os.path.join(FilePath, filename)+'.npy'), ConCated_File)
else:
    File_Name = 'Final_File_'+filename
    File_Array = globals()[File_Name]
    np.save((os.path.join(FilePath, filename)+'.npy'), File_Array)


100%|██████████| 800/800 [1:10:29<00:00,  5.29s/it]


In [ ]:
Halo_Array[np.where(Is_First_Approach)]


array([ 441,  464,  508,  559,  604,  611,  635,  705,  729,  802,  899,
        910,  941, 1017, 1060, 1094, 1114, 1121, 1149, 1190, 1199, 1215,
       1220, 1269, 1273, 1294, 1334, 1356, 1452, 1465, 1486, 1496, 1502,
       1507, 1521, 1553, 1596, 1598, 1606, 1736, 1757, 1765, 1771, 1840,
       1854, 1863, 1889, 1901, 1940, 1950, 1954, 1997, 2058, 2148, 2165,
       2171, 2247, 2252, 2254, 2371, 2400, 2434, 2458, 2483, 2529, 2569,
       2587, 2597, 2671, 2704, 2740, 2858, 2861, 2867, 2923, 2936, 2955,
       2967, 2974, 3008, 3029, 3031, 3106, 3115, 3233, 3277, 3314, 3362,
       3383, 3397, 3527, 3567, 3585, 3598, 3620, 3641, 3718, 3772, 3953,
       3959, 3971, 4065, 4097, 4154, 4157, 4166, 4174, 4184, 4223, 4279,
       4352, 4360, 4363, 4382, 4454, 4470, 4518, 4597, 4638, 4647, 4684,
       4825, 4925])

: 

In [ ]:
Halo_Array[:len(ConCated_File)][ConCated_File]


: 

In [22]:
ConCated_File.shape


(200,)